Import of some liberaries

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore') # To make code prettier and convinient
from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import of NLTK liberary

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Data loading

In [3]:
fake = pd.read_csv('/content/drive/MyDrive/Fake news/Fake.csv')
real = pd.read_csv('/content/drive/MyDrive/Fake news/True.csv')
fake = fake[:1500]
real = real[:1500]

In [4]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1500 non-null   object
 1   text     1500 non-null   object
 2   subject  1500 non-null   object
 3   date     1500 non-null   object
dtypes: object(4)
memory usage: 47.0+ KB


In [5]:

real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1500 non-null   object
 1   text     1500 non-null   object
 2   subject  1500 non-null   object
 3   date     1500 non-null   object
dtypes: object(4)
memory usage: 47.0+ KB


In [6]:
real['class'] = 1
fake['class'] = 0

In [7]:
real["text"] = real['title'] + '' + real['text']
fake["text"] = fake['title'] + '' + fake['text']
real=real.drop(["subject", "date", "title"], axis = 'columns')
fake=fake.drop(["subject", "date", "title"], axis = 'columns')

In [8]:
data = real.append(fake, ignore_index = True)
data.head()

,text,class
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [9]:
data.head()

,text,class
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [10]:
data.isnull().sum()

text     0
class    0
dtype: int64

#### Stemming:

#### Stemming is the process of reducing a word to it's Root word
#### e.g. : actor,actress,acting -> act

In [11]:
porter_stem = PorterStemmer()

In [12]:
def stemming(content):
    stem_content = re.sub('[^a-zA-Z]',' ',content)
    stem_content = stem_content.lower()
    stem_content = stem_content.split()
    stem_content = [porter_stem.stem(word) for word in stem_content if not word in stopwords.words('english')]
    stem_content = ' '.join(stem_content)
    return stem_content

In [13]:
 data['text'] = data['text'].apply(stemming)

In [14]:
data

,text,class
0,u budget fight loom republican flip fiscal scr...,1
1,u militari accept transgend recruit monday pen...,1
2,senior u republican senat let mr mueller job w...,1
3,fbi russia probe help australian diplomat tip ...,1
4,trump want postal servic charg much amazon shi...,1
...,...,...
2995,lawyer trump use prove russia connect russia l...,0
2996,congress demand white hous turn alleg comey ta...,0
2997,trump lawyer say trump russian incom except in...,0
2998,donald trump attack stephen colbert colbert sh...,0


In [15]:
Input = data['text'].values
Output = data['class'].values

Vectorizer:
It is used to get some distinct features out of the text for the model to train on, by converting text to numerical vectors.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
vectorizer = TfidfVectorizer()
vectorizer.fit(Input)
Input = vectorizer.transform(Input)

In [18]:
print(Input)

  (0, 23350)	0.03551060664357594
  (0, 23297)	0.02725402949985217
  (0, 23272)	0.04865431702625214
  (0, 23026)	0.04095237452907235
  (0, 22986)	0.018682888418294675
  (0, 22833)	0.036395513751232636
  (0, 22823)	0.05153510126491726
  (0, 22754)	0.017102298533786336
  (0, 22671)	0.027640613125065733
  (0, 22660)	0.020642414325973232
  (0, 22659)	0.05407869752276488
  (0, 22626)	0.03918628903178879
  (0, 22620)	0.04895425936356346
  (0, 22551)	0.019840503270512972
  (0, 22517)	0.02244907454388816
  (0, 22476)	0.07571352187147412
  (0, 22454)	0.06557676854302766
  (0, 22353)	0.09200566963543264
  (0, 21923)	0.020800507537822862
  (0, 21902)	0.03296701038572832
  (0, 21686)	0.020399556083963707
  (0, 21340)	0.019628413683412563
  (0, 21153)	0.0735691575229767
  (0, 21104)	0.10938673145513549
  (0, 21085)	0.04185347520068469
  :	:
  (2999, 5088)	0.07537295243589523
  (2999, 4840)	0.06761607217245771
  (2999, 4807)	0.03435818332592013
  (2999, 4774)	0.06297044550375995
  (2999, 4483)	0.0676

In [49]:
acc = []
acc_scores={}

Spliting of data:

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
xtrain, xtest, ytrain, ytest = train_test_split(Input, Output, test_size=0.2,random_state=2)

In [52]:
xtest

<600x23582 sparse matrix of type '<class 'numpy.float64'>'
	with 95641 stored elements in Compressed Sparse Row format>

Logistic Regression:

In [53]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000)

In [54]:
model.fit(xtrain,ytrain)

LogisticRegression(max_iter=10000)

In [55]:
ypred = model.predict(xtest)
ypred

array([1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,

In [56]:
model.score(xtrain,ytrain)

0.99625

In [57]:
acc.append(accuracy_score(ypred,ytest))
acc_scores["Logistic Regression"]=acc[0]

In [58]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       308
           1       0.99      0.99      0.99       292

    accuracy                           0.99       600
   macro avg       0.99      0.99      0.99       600
weighted avg       0.99      0.99      0.99       600



Naive Baye's Classifier:


In [59]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()

In [60]:
model1=gnb.fit(xtrain.toarray(),ytrain)

In [61]:
model1.score(xtrain.toarray(),ytrain)

1.0

In [62]:
ypred1=model1.predict(xtest.toarray())

In [63]:
acc.append(accuracy_score(ypred1,ytest))
acc_scores["Naive Baye's Classifier"]=acc[1]

In [64]:
print(classification_report(ytest,ypred1))

              precision    recall  f1-score   support

           0       0.86      0.93      0.90       308
           1       0.92      0.85      0.88       292

    accuracy                           0.89       600
   macro avg       0.89      0.89      0.89       600
weighted avg       0.89      0.89      0.89       600



Decision Tree Classifier:

In [65]:
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()

In [66]:
model2.fit(xtrain,ytrain)

DecisionTreeClassifier()

In [67]:
model2.score(xtrain,ytrain)

1.0

In [68]:
ypred2 = model2.predict(xtest)
acc.append(accuracy_score(ypred2,ytest))
acc_scores["Decision Tree Classifier"]=acc[2]

Random Forest Classifier:

In [69]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier(n_estimators=20, random_state=0)

In [70]:
model3.fit(xtrain,ytrain)

RandomForestClassifier(n_estimators=20, random_state=0)

In [71]:
model3.score(xtrain,ytrain)

1.0

In [72]:
ypred3 = model3.predict(xtest)
acc.append(accuracy_score(ypred3,ytest))
acc_scores["Random Forest Classifier"]=acc[3]

KNeighbors Classifier:

In [73]:
from sklearn.neighbors import KNeighborsClassifier
model4=KNeighborsClassifier()

In [74]:
model4.fit(xtrain,ytrain)

KNeighborsClassifier()

In [75]:
model4.score(xtrain,ytrain)

0.8975

In [76]:
ypred4=model4.predict(xtest)
acc.append(accuracy_score(ypred4,ytest))
acc_scores["KNearestNeighbor Classifier"]=acc[4]

Comparision of Different models:

In [77]:
acc_scores = list(acc_scores.items())

In [78]:
for model,score in acc_scores:
    score=round((score*100),2)
    print(f"Accuracy score of {model} = {score} % ")

Accuracy score of Logistic Regression = 99.17 % 
Accuracy score of Naive Baye's Classifier = 89.0 % 
Accuracy score of Decision Tree Classifier = 99.67 % 
Accuracy score of Random Forest Classifier = 99.0 % 
Accuracy score of KNearestNeighbor Classifier = 87.33 % 


Conclusion:                     
Logistic Regression,Decision Tree Classifier perform better than other models.